# <center>Implementing LDA in Python</center>

<center>Dr. W.J.B. Mattingly</center>

<center>Smithsonian Data Science Lab and United States Holocaust Memorial Museum</center>

<center>February 2021</center>

## Key Concepts in this Notebook

## Introduction

## Importing the Required Libraries

In [18]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
# import pyLDAvis.gensim

import pyLDAvis.gensim_models as gensimvis

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Preparing the Data

In [3]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [4]:
stopwords = stopwords.words("english")

In [5]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
data = load_data("data/ushmm_dn.json")["texts"]

print (data[0][0:90])

 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928. 


In [9]:
url = '/Users/dj/Documents/GitHub/inbraak_'
numbers = np.arange(1,7)
ending = '.txt'

texts = []
for i in numbers:
    p = url + str(i) + ending
    f = open(p,'r')
    text = f.read()
    texts.append(text)

data = texts

In [10]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

Translated(src d en text public prosecutor district prosecutor office suspect bear date bi


In [11]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['translated', 'src', 'en', 'text', 'public', 'prosecutor', 'district', 'prosecutor', 'office', 'suspect', 'bear', 'date', 'birth', 'place', 'birth', 'currently', 'detain', 'detention', 'address', 'counsel']


In [12]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 2), (1, 6), (2, 3), (3, 1), (4, 7), (5, 2), (6, 1), (7, 2), (8, 4), (9, 2), (10, 5), (11, 6), (12, 3), (13, 3), (14, 1), (15, 6), (16, 2), (17, 1), (18, 12), (19, 4)]
above


In [24]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


## Vizualizing the Data

In [25]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.069505  0.032610       1        1  82.084733
0     -0.008190 -0.078536       2        1  17.816763
2     -0.061315  0.045927       3        1   0.098504, topic_info=         Term       Freq      Total Category  logprob  loglift
76      house  39.000000  39.000000  Default  30.0000  30.0000
4     address  37.000000  37.000000  Default  29.0000  29.0000
39      count  21.000000  21.000000  Default  28.0000  28.0000
183  consider  19.000000  19.000000  Default  27.0000  27.0000
186     court  47.000000  47.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
267    report   0.004723  13.667888   Topic3  -6.4912  -1.0476
89    legally   0.004751  16.975862   Topic3  -6.4851  -1.2583
18      break   0.004811  27.432229   Topic3  -6.4726  -1.7257
44    defense   0.004808  30.604975   Topic3  -6.4733  -1.8358
40      crime   0.004795  28.181554   Topic3  -6.4760  -1.7560

[223 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         1  1.017793        above
4         1  0.810143      address
4         2  0.189033      address
307       1  0.687684        again
309       1  1.030740  alternative
...     ...       ...          ...
147       2  0.453955       window
301       2  0.770157   windowpane
302       1  0.683904      witness
302       2  0.284960      witness
303       2  0.770080        write

[199 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3])

In [1]:
# Import Mannes Data
import pandas as pd

In [7]:
url = '/Users/dj/Documents/GitHub/out.csv'

df = pd.read_csv(url)
print (df)

                                                      0
0     Onderzoek van de zaakDit arrest is gewezen naa...
1     TenlasteleggingAan de verdachte is tenlastegel...
2     Vonnis waarvan beroepHet vonnis waarvan beroep...
3     BewijsoverwegingDe raadsman heeft zich ter ter...
4     BewezenverklaringHet hof acht wettig en overtu...
...                                                 ...
7552  6De strafoplegging6.1De vordering van de offic...
7553  7De wettelijke voorschriftenDe beslissing beru...
7554  8De beslissingDe rechtbank:Vrijspraak- spreekt...
7555  9Bijlage IDe tenlastelegginghij op of omstreek...
7556  10Bijlage IIDe bewijsmiddelenAangezien verdach...

[7557 rows x 1 columns]


In [8]:
str(df.iloc[2,])

'0    Vonnis waarvan beroepHet vonnis waarvan beroep...\nName: 2, dtype: object'